In [6]:
######################################## STEP 1 - Data acquistion #######################################################

##### ########## ######### Vehicle complaints Dataset ######################################################

## Downloading the Vehicle Complaints dataset

import requests
import os
import pandas as pd
import zipfile
import numpy as np
from sklearn.preprocessing import LabelEncoder


# URL
VehicleComplaints_dataset_url = "https://static.nhtsa.gov/odi/ffdd/cmpl/FLAT_CMPL.zip"
# Filename for the downloaded dataset in the current directory
filename_vehicle_complaints = "FLAT_CMPL.zip"
# Checking if the file already exists and making a HTTP GET request to the server
if not os.path.exists(filename_vehicle_complaints):
    # Download the dataset
    response = requests.get(VehicleComplaints_dataset_url, stream=True) #HTTP GET
    if response.status_code == 200: 
        with open(filename_vehicle_complaints, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Downloaded the 'Vehicle Complaints' dataset to {filename_vehicle_complaints}")
    else:
        print("Failed to download the dataset.")

# It is a Zip file - Extracting the ZIP file to the current directory
with zipfile.ZipFile(filename_vehicle_complaints, 'r') as zip_ref:
    zip_ref.extractall()
print(f"Extracted contents from {filename_vehicle_complaints} to the current directory")                
Extracted_file = 'FLAT_CMPL.txt'
df = pd.read_csv(Extracted_file, sep='\t', on_bad_lines='skip', low_memory=False , header=None) #TAB separator


#The file is a TXT... we need to convert it to to CSV
csv_file = 'FLAT_CMPL.csv'
df.to_csv(csv_file, index=False)
print(f'TXT file converted to CSV and saved as {csv_file}')

################ Showing the dataset ###############
# Load the dataset
df_VehicleComplaints = pd.read_csv(csv_file, low_memory=False)


# Display the content of each DataFrame
# the dataset has no heard and it has no column name defined
new_column_names = ['CMPLID', 'ODINO', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'CRASH', 'FAILDATE', 'FIRE', 'INJURED',
                   'DEATHS', 'COMPDESC', 'CITY', 'STATE', 'VIN', 'DATEA', 'LDATE', 'MILES', 'OCCURENCES', 'CDESCR', 'CMPL_TYPE',
                   'POLICE_RPT_YN', 'PURCH_DT', 'ORIG_OWNER_YN', 'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'NUM_CYLS',
                   'DRIVE_TRAIN', 'FUEL_SYS', 'FUEL_TYPE', 'TRANS_TYPE', 'VEH_SPEED', 'DOT', 'TIRE_SIZE', 'LOC_OF_TIRE',
                   'TIRE_FAIL_TYPE', 'ORIG_EQUIP_YN', 'MANUF_DT', 'SEAT_TYPE', 'RESTRAINT_TYPE', 'DEALER_NAME',
                   'DEALER_TEL', 'DEALER_CITY', 'DEALER_STATE', 'DEALER_ZIP', 'PROD_TYPE', 'REPAIRED_YN', 'MEDICAL_ATTN',
                   'VEHICLES_TOWED_YN']

###filling tgat dataset with the corresponding column names
df_VehicleComplaints.columns = new_column_names
print("Downloaded Dataset - Vehicle Complaints with corresponding column names")
display(df_VehicleComplaints.head())
#saving
df_VehicleComplaints.to_csv("vehicleComplaints.csv", index=False)


############################################################### STEP 2 - DATA PROCESSING ######################################
######## ############    Remove outliers ################## ####################

print("NULL outliers BEFORE")
columns_make_model = ['MAKETXT','MODELTXT', 'MFR_NAME']
null_counts = df_VehicleComplaints[columns_make_model].isnull().sum()
display(null_counts)

## We dont want the entries where the vehicle MAKE or MODEL are not listed, so we will remove them
##outliers
df_cleaned = df_VehicleComplaints.dropna(subset=['MFR_NAME'])
df_cleaned2 = df_cleaned.dropna(subset=['MAKETXT'])
df_cleaned3 = df_cleaned2.dropna(subset=['MODELTXT'])

df_VehicleComplaints = df_cleaned3

print("NULL outliers AFTER")
columns_make_model = ['MAKETXT','MODELTXT', 'MFR_NAME']
null_counts = df_VehicleComplaints[columns_make_model].isnull().sum()
display(null_counts)

##We also alre not interested in vehicles with the year = 9999. That will be considered an outlier 
#The owners of the dataset use this value when they dont know the year of the vehicle
df_VehicleCompliants_Outliers = df_VehicleComplaints[df_VehicleComplaints['YEARTXT'] != 9999]
print("Clean Dataset -without the entrie with year equal to 9999")
display(df_VehicleCompliants_Outliers.head())


#########   ############# Remove duplicates ##################        ####################
#because our dataset treats null values as NaN's and NaN == NaN will return false
#creating temporary placeholder
df = df_VehicleCompliants_Outliers.fillna('temporary')
#checking for duplaictes 
duplicates = df.duplicated().sum()
print(duplicates)
#returning the dataset to its former Stage
df = df.replace('temporary', np.nan)
print("Dataset with duplicates removed: ")
display(df.head())



###### Null values Handling ############### ######

## Pandas is treating NULLs and NaNs so we will keep them this way


#################################### Step 3 - DATA TRANSFORMATION ##########################################################
#####      ########## Brands with most complaints recorded ####################      #############

## how many Vehicle there are from each brand with complaints recorded? TOP 5? 
grouped_complaints = df.groupby("MAKETXT").agg({"MAKETXT": "count"}).rename(columns={"MAKETXT": "count"})
grouped_complaints = grouped_complaints.sort_values(by="count", ascending=False)
print("Vehicle Manufactures with the most complaints recorded")
display(grouped_complaints.head())
#saving
grouped_complaints.to_csv("brands_with_most_complaints_recorded.csv")


######## ###########  Crash Severity ############# ###########################           
#new column crash_category
df['crash_category'] = 'Safe Crash'  # Default value for all rows
df.loc[df['DEATHS'] > 0, 'crash_category'] = 'Medium Crash'
df.loc[df['DEATHS'] > 5, 'crash_category'] = 'Big Crash'

print("Crash Category column that was added to our Vehicle Complaints dataframe")
crash_category_column = df['crash_category']
display(crash_category_column.head())


##########    ################## LABEL ENCONDING  #################      ##########
label_encoder = LabelEncoder()

# Applying label enconding to our column crash category
df['Crash_category_encoded'] = label_encoder.fit_transform(df['crash_category'])
display(df.head())


########## ######### Changing column datatype #######3  ###########
print("OCCURENCES change of datatype, before: ")
before = df['OCCURENCES'].dtype
display(before)
df['OCCURENCES'] = pd.to_numeric(df['OCCURENCES'], errors='coerce', downcast='float')

print("OCCURENCES change of datatype, after: ")
after = df['OCCURENCES'].dtype
display(after)


############### ####################  Dimension and Factual Tables ########### ###############3
# Create dimension tables
vehicle_dimension = df[['VIN', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'FUEL_SYS', 'TRANS_TYPE', 'DRIVE_TRAIN']].drop_duplicates().reset_index(drop=True).copy()
location_dimension = df[[ 'CITY', 'STATE']].drop_duplicates().reset_index(drop=True).copy()
complaint_type_dimension = df[[  'COMPDESC', 'DEATHS', 'INJURED', 'FIRE' , 'POLICE_RPT_YN']].drop_duplicates().reset_index(drop=True).copy()

# Create the factual table
complaints_factual = df[['CMPLID', 'VIN', 'MAKETXT', 'MODELTXT', 'CITY', 'STATE', 'CDESCR', 'LDATE']].copy()

# Print dimension tables
print("Vehicle Dimension:")
display(vehicle_dimension.head())
print("\nLocation Dimension:")
display(location_dimension.head())
print("\nComplaint Type Dimension:")
display(complaint_type_dimension.head())

# Print the factual table
print("\nComplaints Factual Table:")
display(complaints_factual.head())

#saving
complaints_factual.to_csv("complaints_factual.csv", index=False)
complaint_type_dimension.to_csv("complaints_type_dimension.csv", index=False)
location_dimension.to_csv("location_dimension.csv", index=False)
vehicle_dimension.to_csv("vehicle_dimension.csv", index=False)


################### Saving the file ###################
csv_file = "VehicleComplaints_Transformed.csv"

#to_csv() method to save the DataFrame to a CSV file (header is included by default)
df.to_csv(csv_file, index=False)  # Set index=False to exclude the index column

print("DataFrame Vehicle Complaints saved to CSV file with header.")

Downloaded the 'Vehicle Complaints' dataset to FLAT_CMPL.zip
Extracted contents from FLAT_CMPL.zip to the current directory
TXT file converted to CSV and saved as FLAT_CMPL.csv
Downloaded Dataset - Vehicle Complaints with corresponding column names


,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,...,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN
0,1,958241,"Volvo Car USA, LLC",VOLVO,760,1987.0,N,NaN,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
1,2,958130,Ford Motor Company,FORD,THUNDERBIRD,1992.0,N,19941222.0,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2,3,958132,"Kia America, Inc.",KIA,SEPHIA,1994.0,Y,19941230.0,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
3,4,958133,"Chrysler (FCA US, LLC)",DODGE,600,1987.0,N,19941231.0,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
4,5,958137,"Chrysler (FCA US, LLC)",DODGE,CARAVAN,1991.0,N,19941218.0,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N


NULL outliers BEFORE


MAKETXT     25
MODELTXT    27
MFR_NAME    25
dtype: int64

NULL outliers AFTER


...


Complaint Type Dimension:


,COMPDESC,DEATHS,INJURED,FIRE,POLICE_RPT_YN
0,ENGINE AND ENGINE COOLING:COOLING SYSTEM:RADIA...,0,0,N,N
1,"FUEL SYSTEM, GASOLINE:DELIVERY",0,0,N,N
2,POWER TRAIN:AUTOMATIC TRANSMISSION,0,0,N,N
3,"FUEL SYSTEM, GASOLINE:STORAGE:TANK ASSEMBLY",0,0,N,N
4,SEATS,0,0,N,N



Complaints Factual Table:


,CMPLID,VIN,MAKETXT,MODELTXT,CITY,STATE,CDESCR,LDATE
0,1,NaN,VOLVO,760,EL CAJON,CA,RADIATOR FAILED @ HIGHWAY SPEED OBSTRUCTING DR...,19950103
1,2,1FAPP6045NH,FORD,THUNDERBIRD,CLINTONTOWN,MI,"FUEL LEAKED FROM FUEL TANK AREA, EMITTING STRO...",19950103
2,3,NaN,KIA,SEPHIA,SAN FRANCISC,CA,SHIFTED INTO REVERSE VEHICLE JERKED VIOLENTLY....,19950103
3,4,1B3BE36D4HC,DODGE,600,MUSKEGON,MI,FUEL TANK ; LEAKS BECAUSE OF RUST GAS LEAK BY ...,19950103
4,5,2B4GK4535MR,DODGE,CARAVAN,MESQUITE,TX,"DRIVER SIDE SEAT FRAME BROKE IN TWO, CAUSING S...",19950103


DataFrame Vehicle Complaints saved to CSV file with header.
